In [ ]:
# Requires labelme == 3.16.5, otherwise utils.labelme_shapes_to_label() cannot be called
import json
import csv
import os
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from sklearn.linear_model import LinearRegression
from labelme import utils
file_path = os.getcwd()
file_path

In [ ]:
from matplotlib.font_manager import FontManager
import subprocess
mpl_fonts = set(f.name for f in FontManager().ttflist)
print('all font list get from matplotlib.font_manager:')
for f in sorted(mpl_fonts):
    print('\t' + f)
import matplotlib
plt.rcParams['font.sans-serif'] = ['AR PL UMing CN']
matplotlib.rcParams['axes.unicode_minus'] = False
%matplotlib inline

# Image annotation visualization

In [ ]:
for name in os.listdir(file_path +  f'/labels/json'): 
    m=name[:-5]
    json_path = file_path +  f'/labels/json/{m}.json'
    data = json.load(open(json_path))
    img = io.imread(file_path +  f'/labels/images/{m}.jpg')
    data_labels=[]
    for x in data['shapes'][0:14]:
        if x['label']=='矩形框':
            pass
        else:
            data_labels.append(x)
    lab, lab_names = utils.labelme_shapes_to_label(img.shape, data_labels)
    print(lab_names)
    captions = ['%s' % (name) for l, name in enumerate(lab_names)]
    lab_ok = utils.draw_label(lab, img, captions)
    output_folder = file_path + '/labels/raw_images_annotation'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    output_image_path = os.path.join(output_folder, f'{m}.jpg')
    io.imsave(output_image_path, lab_ok)
    print(f"'{m}.jpg' saved to{output_image_path}")

# change json tag

In [ ]:
import json
def process_single_json(labelme_path, save_folder=file_path + f'/C2/json_fin/'):
    for name in os.listdir(file_path +  f'/labels/json/'): 
        j=name[:-5]
        json_path = file_path +  f'/labels/json/{j}.json'
        with open(json_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        index=[]
        for i in range(len(data['shapes'])):   
            if data['shapes'][i]['label'] == "骨性髋臼凸面最高点":
                data['shapes'][i]['label'] = "骨性髋臼顶凸面最高点"
        with open(json_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=2) # Set ensure_ascii to False
        print(f"标签已修改并保存到'{json_path}'")
# Call the function
process_single_json(file_path + f'/labels/json')

# Calculate the angle from the predicted coordinates

In [ ]:
# function
# '基线点','骨缘点','髂骨下缘点','盂唇中心点','骨性髋臼顶凸面最高点','回声失落点'
# P1, Baseline Point (BP); P2, Bony Rim Point (BRP); P3, Lower Limb Point (LLP); P4, The Center of the Labrum (COP);
# P5, Peak of the Acetabular Prominence (POAP); P6, Echo Loss Point (ELP)
class ab:
    def __init__(self, data, num):
        self.data = data.groupby('Point')
        self.num = num
        self.result_dict = {}
        for group_name, group_data in self.data:
            x_values = group_data['X'].tolist()
            y_values = group_data['Y'].tolist()
            self.result_dict[group_name] = list(zip(x_values, y_values))
    def angle(self):
        angle_alpha = []
        angle_beta = []
        for i in self.num:
            label = f'img{i}'
            x1 = list(self.result_dict[label][0])
            x2 = list(self.result_dict[label][1])
            p1 = np.array([x1[0], x1[1]])  # 基线点
            p2 = np.array([x2[0], x2[1]])  # 骨缘点
            x1 = list(self.result_dict[label][3])
            p4 = np.array([x1[0], x1[1]])  # 盂唇中心点
            x1 = list(self.result_dict[label][2])
            x2 = list(self.result_dict[label][4])
            p5 = np.array([x2[0], x2[1]])  # 骨性髋臼顶凸面最高点
            p3 = np.array([x1[0], x1[1]])  # 髂骨下缘点
            x3 = list(self.result_dict[label][5]) 
            p6 = np.array([x3[0],x3[1]])   # 回声失落点
            X = np.array([p1[0], p2[0]]).reshape(-1, 1)
            Y = np.array([p1[1], p2[1]]).reshape(-1, 1)
            lin_reg.fit(X, Y)
            jixian_k = lin_reg.coef_[0][0]
            jixian_b = lin_reg.intercept_[0]
            # beta
            X = np.array([p4[0], p6[0]]).reshape(-1, 1)
            Y = np.array([p4[1], p6[1]]).reshape(-1, 1)
            lin_reg.fit(X, Y)
            beta_k = lin_reg.coef_[0][0]
            beta_b = lin_reg.intercept_[0]
            # alpha
            X = np.array([p5[0], p3[0]]).reshape(-1, 1)
            Y = np.array([p5[1], p3[1]]).reshape(-1, 1)
            lin_reg.fit(X, Y)
            alpha_k = lin_reg.coef_[0][0]
            alpha_b = lin_reg.intercept_[0]
            alpha = calculate_angle(alpha_k,alpha_b,jixian_k,jixian_b)
            beta = calculate_angle(beta_k,beta_b,jixian_k,jixian_b)
            angle_alpha.append(alpha)
            angle_beta.append(beta)
        return angle_alpha, angle_beta
def calculate_angle(k1,b1,k2,b2):
    slope_difference = (k2 - k1)/(1 + k1*k2)
    angle_rad = math.atan(abs(slope_difference))
    angle_deg = math.degrees(angle_rad)
    return angle_deg
def calculate_accuracy(X, Y):
    accuracy=[]
    if len(X) != len(Y):
        raise ValueError("Input lists must have the same length.")
    for i in np.arange(3,11,1):
        correct_count = sum(1 for x, y in zip(X, Y) if abs(x - y) <= i)
        total_count = len(X)
        accuracy.append(correct_count / total_count * 100)
    return accuracy
def read_images_from_folder(folder_path):
    images = []
    for filename in folder_path:
        image = Image.open(filename)
        images.append(image)
    return images
def json_to_numpy(dataset_path):
    with open(dataset_path,  'r', encoding='utf8') as fp:
        json_data = json.load(fp)
        points = json_data['shapes']
    landmarks = []
    for point in points:
        if point['label'] == '基线点':
            for p in point['points']:
                landmarks.append(p)
    for point in points:
        if point['label'] == '骨缘点':
            for p in point['points']:
                landmarks.append(p)
    for point in points:
        if point['label'] == '髂骨下缘点':
            for p in point['points']:
                landmarks.append(p)
    for point in points:
        if point['label'] == '盂唇中心点':
            for p in point['points']:
                landmarks.append(p)
    for point in points:
        if point['label'] == '骨性髋臼顶凸面最高点':
            for p in point['points']:
                landmarks.append(p)
    for point in points:
        if point['label'] == '回声失落点':
            for p in point['points']:
                landmarks.append(p)
    print(landmarks)            
    assert len(landmarks)==6
    print(landmarks)
    landmarks = np.array(landmarks)
    landmarks = landmarks.reshape(-1, 2)
    points=[]
    for x in landmarks:
        points.append(tuple(x))
    return points

In [ ]:
dict={}
Name=[]
for name in os.listdir(file_path +  f'/labels/json'): 
    i=name[:-5]
    Name.append(i)
    json_path = file_path +  f'/labels/json/{i}.json'
    print(i)
    point = json_to_numpy(json_path)
    img=f'img{i}'
    dict[img]=point
output_file = file_path+f'/grouth.csv'
with open(output_file, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Point', 'X', 'Y'])
    for m, tensor in dict.items():
        for j, (x, y) in enumerate(tensor):
            csv_writer.writerow([m, x, y])
data_gro=pd.read_csv(file_path+'/grouth.csv')
X2=data_gro['X'].values
Y2=data_gro['Y'].values
b=ab(data_gro,Name)
a2,b2=b.angle()
result_df = pd.DataFrame({'name':Name,'a1': a2, 'b1': b2})
result_df.to_csv('ab.csv',index=False)